In [1]:
import time
import multiprocessing
from tqdm.notebook import tqdm
from itertools import repeat


In [2]:
def foobar(idx, lock):
    for i in range(0, 4):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx

In [10]:
class MultiprocessManager():

    def __init__(self, num_workers = 4):
        # Create a pointer for a progress bar to track progress
        self.progress_bar = None
        
        # Create a lock to manage internal access to the progress bar
        self.manager = multiprocessing.Manager()
        self.lock = self.manager.Lock()
        
        # Create internal pool to manage processes
        self.pool = multiprocessing.Pool(processes=4)
        
        # Create a shared memory object to store the number of completed processes
        # between the various processes running in parallel
        self.shared_memory_name = "multiprocessor"
        try:
            self.shared_memory = multiprocessing.shared_memory.SharedMemory(name=self.shared_memory_name, create=True, size=1)
        except FileExistsError as fee:
            self.shared_memory = multiprocessing.shared_memory.SharedMemory(name=self.shared_memory_name, create=False, size=1)
        self.shared_memory_buffer = self.shared_memory.buf
        self.shared_memory_buffer[0] = 0

    def _create_progress_bar(self, num_ops):

        return tqdm(range(num_ops))
        
    def _update_progress_bar(self, i):
        self.progress_bar.last_print_n = i
        self.progress_bar.n = i
        self.progress_bar.set_description(f'Completed Ops')
        self.progress_bar.refresh()

    # To make things simple we will define an internal function to hide the details
    # of the progress bar. This way the user can wwrite functions without worrying
    # about updating or locking the progress bar.
    @staticmethod
    def _wrapper_function(func, args, kwargs, internal_params):

        # Extract internal arguments for the manager's internal functions
        internal_args, internal_kwargs = internal_params
        lock, shared_memory_name = internal_args
        
        # Create pointer to shared memory (so we can update the progress bar)
        sm = multiprocessing.shared_memory.SharedMemory(name=shared_memory_name, create=False, size=1)
        smb = sm.buf

        # Run the user specificed function
        try:
            return func(*args, **kwargs)
        
        # Update the completed process counter in the shared memory buffer
        finally:
            lock.acquire()
            smb[0] = int(smb[0]) + 1
            lock.release()       

    # This function will run a set of functions in separate processes.
    # The main thread will block and update a progress bar as the functions complete
    # It will return a result set once all the processes have finished or an
    # error has been encountered.
    def parralelize(self, func, arg_set, kwarg_set):
        
        # Setup vars to help kick off parallelization
        num_ops = len(arg_set)
        self.progress_bar = self._create_progress_bar(num_ops)
        self.shared_memory_buffer[0] = 0
        
        # Configure the parameters for the parallelization
        func_set = [func for i in range(0, num_ops)]
        internal_param_set = [([self.lock, self.shared_memory_name], {}) for i in range(0, num_ops)]   
        param_set = zip(func_set, arg_set, kwarg_set, internal_param_set)
        
        # Start the functions in separate parallel processes and don't block
        apply_result = self.pool.starmap_async(self._wrapper_function, param_set)
        
        # Wait for the proceses to finish while updating the progress bar
        while not apply_result.ready():
            self._update_progress_bar(self.shared_memory_buffer[0])
            time.sleep(0.5)
        self._update_progress_bar(self.shared_memory_buffer[0])

        # Return the results
        return apply_result

In [11]:
num_ops = 10

m = multiprocessing.Manager()
lock = m.Lock()

my_func = foobar
my_func_arg_set = list(zip(list(range(0, num_ops)), repeat(lock)))
my_func_kwarg_set = [{} for i in range(0, num_ops)]

mp = MultiprocessManager(num_workers=1)
apply_result = mp.parralelize(my_func, my_func_arg_set, my_func_kwarg_set)

  0%|          | 0/10 [00:00<?, ?it/s]

0 -> 0
1 -> 0
2 -> 0
3 -> 0
0 -> 1
1 -> 1
2 -> 1
3 -> 1
0 -> 2
1 -> 2
2 -> 2
3 -> 2
0 -> 3
1 -> 3
2 -> 3
3 -> 3
4 -> 0
5 -> 0
6 -> 0
7 -> 0
4 -> 1
5 -> 1
6 -> 1
7 -> 1
4 -> 2
5 -> 2
6 -> 2
7 -> 2
4 -> 3
5 -> 3
6 -> 3
7 -> 3
8 -> 0
9 -> 0
8 -> 1
9 -> 1
8 -> 2
9 -> 2
8 -> 3
9 -> 3
